In [1]:
import pandas as pd
import numpy as np

from scipy import linalg
from scipy import stats

In [15]:
# Load data for change detection as a pandas data frame.
infys_df =  pd.read_csv("../data/INFyS_Selection.csv")
infys_df.head()

,Unnamed: 0,Plot_ID,File,Conglomerado,Sitio,Anio,species_count,total_entries,H,J,AvgTreeHeight,AvgDbh,AvgCrownDiameter,AvgCrownHeight,AvgCrownArea,X,Y
0,1,1_60_1_2005,1,60,1,2005,1,1,0.000000,NaN,3.200000,8.300000,5.000000,NaN,19.635000,-116.349722,32.515556
1,2,1_64_4_2005,1,64,4,2005,1,1,0.000000,NaN,5.500000,14.000000,2.500000,NaN,4.908750,-116.139861,32.532860
2,3,1_153_1_2005,1,153,1,2005,3,14,0.898205,0.817582,3.885714,13.864286,2.442308,2.993077,5.953181,-116.076416,32.448305
3,4,1_153_2_2005,1,153,2,2005,2,6,0.450561,0.650022,4.116667,15.350000,2.966667,3.191667,8.217902,-116.076388,32.448694
4,5,1_153_3_2005,1,153,3,2005,2,8,0.376770,0.543564,4.187500,14.125000,2.925000,3.387500,7.824547,-116.075999,32.448083


In [16]:
# Create new ids by concatenating 'Conglomerado' and 'Sitio'.
infys_df['cng_sit'] = infys_df['Conglomerado'].astype(str) + '_' + infys_df['Sitio'].astype(str)

# As we want to compare cycle 1 vs cycle 2 lets separate them.
infys_df_c1 = infys_df[infys_df['File'] == 1]
infys_df_c2 = infys_df[infys_df['File'] == 2]

In [20]:
# Find intersection of new ID variable: 'cng_sit', so obs. that are present in both cycles.
s1 = infys_df_c1['cng_sit']
s2 = infys_df_c2['cng_sit']
cng_sit_inter = pd.Series(list(set(s1).intersection(set(s2))))

In [22]:
# Filter out obs. that dont appear in cng_sit_inter.
infys_df_c1 = infys_df_c1[infys_df_c1['cng_sit'].isin(cng_sit_inter)]
infys_df_c2 = infys_df_c2[infys_df_c2['cng_sit'].isin(cng_sit_inter)]

In [53]:
# Select only variables to be utilized in change detection.
vars = ["H", "J", "AvgTreeHeight", "AvgDbh", "AvgCrownDiameter", "AvgCrownHeight", "AvgCrownArea"]
nvars = len(vars)
infys_df_c1_c = infys_df_c1[infys_df_c1.columns.intersection(vars)]
infys_df_c2_c = infys_df_c2[infys_df_c2.columns.intersection(vars)]

c1_c_nans = infys_df_c1_c.isna().any(axis=1).to_numpy()
c2_c_nans = infys_df_c2_c.isna().any(axis=1).to_numpy()

In [62]:
# Creat change data matrix, missing values mask and initial change weights.
dm = np.zeros((2 * nvars, infys_df_c1_c.shape[0]))
dm[0:nvars] = np.transpose(infys_df_c1_c.to_numpy())
dm[nvars:] = np.transpose(infys_df_c2_c.to_numpy())

nodataidx = c1_c_nans | c2_c_nans
gooddataidx = nodataidx == False
dm = dm[:, gooddataidx]
ngood = np.sum(gooddataidx)

[1. 1. 1. ... 1. 1. 1.]


In [68]:
# Change detection iterations.
# iteration of MAD   
wt = np.ones(int(ngood))
delta = 1.0
oldrho = np.zeros(nvars)
iter = 0
 
while (delta > 0.01) and (iter < 50): 
    print(iter)
    # Weighted covariance matrices and means.
    sumw = np.sum(wt)
    means = np.average(dm,axis=1, weights=wt)
    dmc = dm - means[:,np.newaxis]
    dmc = np.multiply(dmc,np.sqrt(wt))
    sigma = np.dot(dmc,dmc.T)/sumw
   
    s11 = sigma[0:nvars, 0:nvars]
    s22 = sigma[nvars:, nvars:]
    s12 = sigma[0:nvars, nvars:]
    s21 = sigma[nvars:, 0:nvars]
    
    # Solution of generalized eigenproblems.
    aux_1 = linalg.solve(s22, s21)
    lama, a = linalg.eig(np.dot(s12, aux_1), s11)
    aux_2 = linalg.solve(s11, s12)
    lamb, b = linalg.eig(np.dot(s21, aux_2), s22)
    
    # Sort a.  
    idx = np.argsort(lama)
    a = a[:, idx]
    
    # Sort b.        
    idx = np.argsort(lamb)
    b = b[:, idx]
    
    # Canonical correlations.        
    rho = np.sqrt(np.real(lamb[idx]))
    
    # Normalize dispersions.  
    tmp1 = np.dot(np.dot(a.T,s11), a)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    a = np.multiply(a, tmp3)
    b = np.mat(b)
    tmp1 = np.dot(np.dot(b.T,s22), b)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    b = np.multiply(b, tmp3)
        
    # Assure positive correlation
    tmp = np.diag(np.dot(np.dot(a.T,s12), b))
    b = np.dot(b,np.diag(tmp / np.abs(tmp)))

    # Canonical and MAD variates
    U = np.dot(a.T , (dm[0:nvars, :] - means[0:nvars, np.newaxis]))    
    V = np.dot(b.T , (dm[nvars:, :] - means[nvars:, np.newaxis]))          
    MAD = U - V  
        
    # New weights.        
    var_mad = np.tile(np.mat(2 * (1 - rho)).T, (1, ngood))    
    chisqr = np.sum(np.multiply(MAD, MAD) / var_mad, 0)
    wt = np.squeeze(1 - np.array(stats.chi2._cdf(chisqr, nvars)))
        
    # Continue iteration.        
    delta = np.sum(np.abs(rho - oldrho))
    oldrho = rho
    iter += 1
    
# reshape to original image size, by including nodata pixels    
MADout = np.zeros((int(nvars + 1), infys_df_c1_c.shape[0]))
MADout[0:nvars, gooddataidx] = MAD
MADout[nvars:(nvars + 1), gooddataidx] = chisqr

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [70]:
np.savetxt("iMAD_results.csv", np.transpose(MADout), delimiter=",")